In [1]:
import json
import os, glob
import numpy as np
import datetime


motion_management_result_path = '/mnt/P/TERAPI/MRLINAC/QA/RTQADATA/MotionManagement'
patient_data_root = r'/mnt/P/TERAPI/MRLINAC/QA/RTQADATA/PATIENT_DATA'
patient_data_archive_root = f'/mnt/P/TERAPI/MRLINAC/QA/RTQADATA/Patient_Data_Archive'
logfile_path = '/mnt/P/TERAPI/MRLINAC/QA/RTQADATA/Logfiles'
logfile_path_archive = f'/mnt/P/TERAPI/MRLINAC/QA/RTQADATA/Logfiles_archive'


In [ ]:
def read_report(filename):
    with open(filename, 'r') as file:
        report = json.load(file)
    return report

def find_start_index(cine_times, treatment_time):
    """
    Find the index in cine_times where the treatment time starts.
    if cine times are longer than treatment times, assume that the treatment is during the end of the cines.
    param cine_times: The list of cine times.
    param treatment_time: The treatment time.
    returns: The index where the treatment time starts.
    """
    if np.max(cine_times) > treatment_time:
        time_discard = np.max(cine_times) - treatment_time
        transversal_start_index = np.argmax(cine_times >= time_discard) 
    else:
        transversal_start_index = 0 
    
    return transversal_start_index


filenames = glob.glob(os.path.join(motion_management_result_path, '*.json'))
reports = [read_report(filename) for filename in filenames]
reports = {(report['PatientID'], report['PlanLabel']): report for report in reports}  




## Filter out reports of interest 
- After 2024-10-01
- Logfile must exist (if treatment was aborted then not there)
- Motion trace should be 99% the length of the treatment time

In [ ]:
from ast import literal_eval
#
# Read treatment info and convert to tuples from strings and then to datetime and floats 
#
treatment_info_str = json.load(open('treatment_info.json','r'))
treatment_info = {literal_eval(key): literal_eval(value) for key,value in treatment_info_str.items()} 
print(len(treatment_info))

def foo(value):
    return (datetime.datetime.strptime(value[0], '%Y-%m-%d').date(), float(value[1]))
treatment_info = {key: foo(value) for key, value in treatment_info.items()} 
print(len(treatment_info))

#
# Filter all treatments before 2024-10-01 
#
treatment_info = { key: value for key, value in treatment_info.items() if value[0] >= datetime.date(2024, 10, 1)} 
print(len(treatment_info))

#
# Filter out motion traces signinifcantly short than treatment times
#
#treatment_info = { key: value for key, value in treatment_info.items() if value[1] > 400} 
#print(len(treatment_info))
for key, value in treatment_info.items():
    patient_ID, plan_label = key
    treatment_time = value[1]
    cine_time = np.max(reports[key]['TimesTransversal'])
    if cine_time < (treatment_time * 0.9):
        print(f"Will remove {key} with cine time {cine_time} and treatment time {treatment_time}")
        

def not_short_cine_times(key, value, reports):
    """
    Remove entries from treatment_info where the cine time is significantly shorter than the treatment time.
    """
    treatment_time = value[1]
    cine_time = np.max(reports[key]['TimesTransversal'])
    if cine_time < (treatment_time * 0.99):
        print(f"Remooooooving {key} with cine time {cine_time} and treatment time {treatment_time}")
        return False
    return True

print(len(treatment_info), 'sdfsdf')    
treatment_info = { key:value for key,value in treatment_info.items() if not_short_cine_times(key, value, reports)}

print(len(treatment_info), 'sdfsdf')
for key, value in treatment_info.items():
    treatment_time = value[1]
    cine_time = np.max(reports[key]['TimesTransversal'])
    if cine_time < (treatment_time * 0.99):
        print(f"Removing {key} with cine time {cine_time} and treatment time {treatment_time}")
        not_short_cine_times(key, value, reports)



465
465
195
Will remove ('194202130656', '12MRL1ADT05') with cine time 439 and treatment time 534.64
Will remove ('194605187592', '10MRL1ADT07') with cine time 287 and treatment time 374.68
195 sdfsdf
Remooooooving ('194202130656', '12MRL1ADT05') with cine time 439 and treatment time 534.64
Remooooooving ('194605187592', '10MRL1ADT07') with cine time 287 and treatment time 374.68
193 sdfsdf


 ## Write the filtered reports to disk

In [ ]:
with open('treatment_late_info.json','w') as f:
    json.dump(treatment_info, f, indent=4)


